In [21]:
import pandas as pd 
import gzip 
from camo import greedy
from matplotlib import pyplot as plot
from datetime import datetime

In [93]:
def parse(path):
    df = {} 
    i = 0
    g = open(path,"r")
    for l in g: 
        d = eval(l)
        df[i] = [d["reviewerID"], d["asin"], datetime.strptime(d["reviewTime"], "%m %d, %Y").isocalendar()[1]+datetime.strptime(d["reviewTime"], "%m %d, %Y").year*100] 
        i += 1 
    return pd.DataFrame.from_dict(df, orient='index',) 
reviews = parse('reviews_Beauty_5.json')
reviews.columns= ["reviewerID", "asin", "timestep"]
reviews.to_csv("Amazon Review Data Edge List.csv")

In [94]:
indexer = dict()
counter =0
for i in reviews.reviewerID.unique():
    indexer[i] = counter
    counter +=1

counter *= 2
for i in reviews.asin.unique():
    indexer[i] = counter
    counter+=1
remapped_orig = []
remapped_dest = []

for row in reviews.iterrows():
    row = row[1]
    remapped_orig.append(indexer[row[0]])
    remapped_dest.append(indexer[row[1]])
remapped = pd.DataFrame.from_dict({"remappedOrig":remapped_orig, "remappedDest":remapped_dest})
remapped = pd.concat([reviews, remapped],  axis=1)
remapped.to_csv("Amazon Review Data Edge List.csv")

In [95]:
counts = reviews.groupby(['reviewerID',"timestep"]).size().reset_index(name='counts')
print(counts["counts"].mean())
print(counts["counts"].std())

1.77446230311
1.59251052931


In [96]:
counts = reviews.groupby(['asin',"timestep"]).size().reset_index(name='counts')
print(counts["counts"].mean())
print(counts["counts"].std())

1.34812520799
1.49526096071


In [ ]:
# product to product 
product_to_product = pd.merge(remapped, remapped, left_on=['reviewerID', 'timestep'], right_on=['reviewerID', 'timestep'])[["asin_x", "asin_y", "timestep", "remappedDest_x", "remappedDest_y"]]
product_to_product.to_csv("Amazon Review Data Edge List Product to Product.csv")
product_to_product.head()

In [ ]:
user_to_user = pd.merge(remapped, remapped, left_on=['asin', 'timestep'], right_on=['asin', 'timestep'],)[["reviewerID_x", "reviewerID_y","asin", "timestep", "remappedOrig_x", "remappedOrig_y"]]
user_to_user.to_csv("Amazon Review Data Edge List User to User.csv")
user_to_user.head()